In [75]:
from math import floor

#######################################################################
# Following are some utilities for tile coding from Rich.
# To make each file self-contained, I copied them from
# http://incompleteideas.net/tiles/tiles3.py-remove
# with some naming convention changes
#
# Tile coding starts
class IHT:
    "Structure to handle collisions"
    def __init__(self, size_val):
        self.size = size_val
        self.overfull_count = 0
        self.dictionary = {}

    def count(self):
        return len(self.dictionary)

    def full(self):
        return len(self.dictionary) >= self.size

    def get_index(self, obj, read_only=False):
        d = self.dictionary
        if obj in d:
            return d[obj]
        elif read_only:
            return None
        size = self.size
        count = self.count()
        if count >= size:
            if self.overfull_count == 0: print('IHT full, starting to allow collisions')
            self.overfull_count += 1
            return hash(obj) % self.size
        else:
            d[obj] = count
            return count

def hash_coords(coordinates, m, read_only=False):
    if isinstance(m, IHT): return m.get_index(tuple(coordinates), read_only)
    if isinstance(m, int): return hash(tuple(coordinates)) % m
    if m is None: return coordinates

def tiles(iht_or_size, num_tilings, floats, ints=None, read_only=False):
    """returns num-tilings tile indices corresponding to the floats and ints"""
    if ints is None:
        ints = []
    qfloats = [floor(f * num_tilings) for f in floats]
    tiles = []
    for tiling in range(num_tilings):
        tilingX2 = tiling * 2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append((q + b) // num_tilings)
            b += tilingX2
        coords.extend(ints)
        tiles.append(hash_coords(coords, iht_or_size, read_only))
    return tiles
# Tile coding ends
#######################################################################

In [17]:
import numpy as np
alpha = 0.6
I = np.identity(5)
i = np.linalg.inv(alpha**(-1)*I)
v = np.array((1,2,4,5,6))
print(v.T@i,alpha**(-1)*I)
np.linalg.det(alpha**(-1)*I)

[0.6 1.2 2.4 3.  3.6] [[1.66666667 0.         0.         0.         0.        ]
 [0.         1.66666667 0.         0.         0.        ]
 [0.         0.         1.66666667 0.         0.        ]
 [0.         0.         0.         1.66666667 0.        ]
 [0.         0.         0.         0.         1.66666667]]


12.860082304526749

In [18]:
(1/0.6)**5

12.860082304526752

In [11]:
import numpy as np
x = [-2.1, -1,  4.3]

y = [3,  1.1,  0.12]

X = np.stack((x, y), axis=1)

S = np.cov(X)
# S
u = np.array([1,0,0])
u.T@S@u

13.004999999999999

In [10]:
import torch
import torch.nn as nn
class NeuralNetwork(nn.Module):

    def __init__(self, inputs, outputs, num_hidden=128):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputs, num_hidden),  # hidden layer
            nn.ReLU(),
            nn.Linear(num_hidden, outputs)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

nn = NeuralNetwork(2,2)
type(nn) is NeuralNetwork
# nn(torch.FloatTensor([1,1]))

True

In [12]:
from itertools import count
for t in count():
    print(t)
    if t > 10:
        break

0
1
2
3
4
5
6
7
8
9
10
11


In [7]:
False+1

1

In [20]:

torch.zeros((1,))

tensor([0.])

In [36]:

# nn.train()
# nn(torch.tensor([0], dtype=torch.float)[None])


tensor([[-0.3404,  0.5700]], grad_fn=<AddmmBackward0>)
<class 'collections.OrderedDict'>


LSTM

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
print(inputs)

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

[tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])]
tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward0>))


In [42]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [58]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [63]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
# print(model.word_embeddings(prepare_sequence(training_data[0][0], word_to_ix)).view(len(prepare_sequence(training_data[0][0], word_to_ix)), 1, -1))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.6999, -1.3512, -1.4088],
        [-0.6782, -1.3561, -1.4490],
        [-0.7557, -1.2753, -1.3824],
        [-0.6392, -1.3967, -1.4923],
        [-0.6397, -1.4038, -1.4833]])
tensor([[-0.6999, -1.3512, -1.4088],
        [-0.6782, -1.3561, -1.4490],
        [-0.7557, -1.2753, -1.3824],
        [-0.6392, -1.3967, -1.4923],
        [-0.6397, -1.4038, -1.4833]], grad_fn=<LogSoftmaxBackward0>) tensor([0, 1, 2, 0, 1])
tensor([[-0.7142, -1.3048, -1.4305],
        [-0.6326, -1.3505, -1.5622],
        [-0.6487, -1.3731, -1.4963],
        [-0.7064, -1.3190, -1.4306]], grad_fn=<LogSoftmaxBackward0>) tensor([1, 2, 0, 1])
tensor([[-0.7317, -1.2988, -1.4023],
        [-0.7135, -1.2992, -1.4383],
        [-0.7917, -1.2243, -1.3745],
        [-0.6730, -1.3374, -1.4814],
        [-0.6721, -1.3453, -1.4746]], grad_fn=<LogSoftmaxBackward0>) tensor([0, 1, 2, 0, 1])
tensor([[-0.7457, -1.2559, -1.4239],
        [-0.6667, -1.2947, -1.5482],
        [-0.6790, -1.3205, -1.4877],
        [-0.7445, -1

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [147]:
a = torch.FloatTensor([1,2,3,4,5,6])
# list(torch.split(a,3))
# a.chunk(2)
# torch.vstack((torch.tensor([1,2,3]),torch.tensor([6,5,4]))).reshape(-1)
# torch.permute(a.view(2,3), (1, 0))
torch.permute(a.view(1,6), (1, 0))


tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.]])

In [122]:
x = torch.rand(5,6)
print(x)
print(x.view(5,2,-1))

tensor([[0.3329, 0.0201, 0.7266, 0.5547, 0.4013, 0.6619],
        [0.5853, 0.6093, 0.8813, 0.2253, 0.4563, 0.9238],
        [0.4590, 0.5134, 0.8083, 0.5445, 0.4613, 0.7685],
        [0.6746, 0.0142, 0.6935, 0.4052, 0.5869, 0.5324],
        [0.8313, 0.0654, 0.7074, 0.7636, 0.1319, 0.4456]])
tensor([[[0.3329, 0.0201, 0.7266],
         [0.5547, 0.4013, 0.6619]],

        [[0.5853, 0.6093, 0.8813],
         [0.2253, 0.4563, 0.9238]],

        [[0.4590, 0.5134, 0.8083],
         [0.5445, 0.4613, 0.7685]],

        [[0.6746, 0.0142, 0.6935],
         [0.4052, 0.5869, 0.5324]],

        [[0.8313, 0.0654, 0.7074],
         [0.7636, 0.1319, 0.4456]]])


In [8]:
def f123():
    yield 1
    yield 2
    yield 3

for item in f123():
    print(item)

1
2


In [12]:
lst=['2','3','']
for idx in range(len(lst)):
    with lst as lst:
        int(lst(idx))
        print(type(lst(idx)))


AttributeError: __enter__

In [3]:
# creating an empty list
lst = []

# number of elements as input
n = int(input("Enter number of elements : "))

# iterating till the range
for i in range(0, n):
    ele = int(input())

    lst.append(ele) # adding the element

print(lst)

ValueError: invalid literal for int() with base 10: ''

In [20]:
import random
random.randint(0,9)

4